In [415]:
import numpy as np
import tensorflow as tf

In [416]:
import tensorflow_datasets as tfds

In [417]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info = True, as_supervised=True)

W0902 17:16:37.687247 76716 dataset_builder.py:439] Warning: Setting shuffle_files=True because split=TRAIN and shuffle_files=None. This behavior will be deprecated on 2019-08-06, at which point shuffle_files=False will be the default for all splits.


In [418]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [419]:
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)
num_validation_samples

<tf.Tensor: id=4115538, shape=(), dtype=int64, numpy=6000>

In [420]:
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)
num_test_samples

<tf.Tensor: id=4115541, shape=(), dtype=int64, numpy=10000>

In [421]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

In [422]:
mnist_info

tfds.core.DatasetInfo(
    name='mnist',
    version=1.0.0,
    description='The MNIST database of handwritten digits.',
    urls=['https://storage.googleapis.com/cvdf-datasets/mnist/'],
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann. lecun. com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
    redistribution_info=,
)

In [423]:
scaled_train_and_validation_data = mnist_train.map(scale)

In [424]:
test_data = mnist_test.map(scale)

In [425]:
BUFFER_SIZE = 10000

shuffle_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffle_train_and_validation_data.take(num_validation_samples)
train_data = shuffle_train_and_validation_data.skip(num_validation_samples)

In [426]:
BATCH_SIZE = 500
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

In [427]:
validation_inputs, validation_targets = next(iter(validation_data))

## Model

In [428]:
inpit_size = 784
output_size = 10
hidden_layer_size = 250

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax') 
])

### Choose the optimizer and the loss function

In [429]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

### Train

In [430]:
NUM_EPOCHS = 10

model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/10
108/108 - 9s - loss: 0.5700 - accuracy: 0.8159 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
108/108 - 8s - loss: 0.1505 - accuracy: 0.9557 - val_loss: 0.1286 - val_accuracy: 0.9628
Epoch 3/10
108/108 - 9s - loss: 0.1030 - accuracy: 0.9685 - val_loss: 0.1008 - val_accuracy: 0.9707
Epoch 4/10
108/108 - 8s - loss: 0.0792 - accuracy: 0.9761 - val_loss: 0.0850 - val_accuracy: 0.9755
Epoch 5/10
108/108 - 8s - loss: 0.0594 - accuracy: 0.9819 - val_loss: 0.0823 - val_accuracy: 0.9772
Epoch 6/10
108/108 - 7s - loss: 0.0548 - accuracy: 0.9826 - val_loss: 0.0596 - val_accuracy: 0.9835
Epoch 7/10
108/108 - 7s - loss: 0.0397 - accuracy: 0.9882 - val_loss: 0.0514 - val_accuracy: 0.9852
Epoch 8/10
108/108 - 8s - loss: 0.0413 - accuracy: 0.9879 - val_loss: 0.0405 - val_accuracy: 0.9895
Epoch 9/10
108/108 - 7s - loss: 0.0348 - accuracy: 0.9895 - val_loss: 0.0426 - val_accuracy: 0.9863
Epoch 10/10
108/108 - 7s - loss: 0.0307 - accuracy: 0.9906 - val_loss: 0.0460 - val_accuracy

## Test

In [434]:
test_loss, test_accuracy = model.evaluate(test_data)
# print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))
test_accuracy


      1/Unknown - 1s 923ms/step - loss: 0.1129 - accuracy: 0.972 - 1s 928ms/step - loss: 0.1129 - accuracy: 0.9722

0.9722